## Exercises in KMeans clustering with Spark and Scala

Based on material from Advanced Analytics with Spark: Ryza, Laserson, Owen & Wills Chapter 5

Uses Almond Scala kernel for Jupyter notebooks. 
Scala version 2.12

Data set is the KDD Cup 1999 Data Set available at http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
The full dataset (c. 740MB was used)

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Logger,Level}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Logger,Level}


In [3]:
val spark = {
    org.apache.spark.sql.NotebookSparkSession.builder().master("local[*]").config("spark.sql.shuffle.partitions","8").getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@3fe7acf6

In [4]:
val dataNoHeader = spark.read
            .option("inferSchema","true")
            .option("header","false")
            .csv("../KDDCup1999_dataset/kddcup.data")

csv at cmd3.sc:4

1 / 1

csv at cmd3.sc:4

8 / 8

dataNoHeader: DataFrame = [_c0: int, _c1: string ... 40 more fields]

In [5]:
dataNoHeader.columns.length

res4: Int = 42

Get the list of headers from the kddcup.names file and use to convert to a DF

In [6]:
val names = spark.read.option("header","false").csv("../KDDCup1999_dataset/kddcup.names")

csv at cmd5.sc:1

1 / 1

names: DataFrame = [_c0: string, _c1: string ... 21 more fields]

In [7]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._

In [8]:
names.createOrReplaceTempView("temp1")

In [9]:
import org.apache.spark.sql.types.DataTypes._

import org.apache.spark.sql.types.DataTypes._

Trim the text after the : to get just the names

In [10]:
val nameDF = spark.sql("select left(_c0, instr(_c0,':')-1) as text from temp1 where instr(_c0,':')>0 union all select 'label'")
val nameArray = nameDF.collect.map(row=>row.getString(0))
nameArray.length

collect at cmd9.sc:2

2 / 2

nameDF: DataFrame = [text: string]
nameArray: Array[String] = Array(
  "duration",
  "protocol_type",
  "service",
  "flag",
  "src_bytes",
  "dst_bytes",
  "land",
  "wrong_fragment",
  "urgent",
  "hot",
  "num_failed_logins",
  "logged_in",
  "num_compromised",
  "root_shell",
  "su_attempted",
  "num_root",
  "num_file_creations",
  "num_shells",
  "num_access_files",
  "num_outbound_cmds",
  "is_host_login",
  "is_guest_login",
  "count",
  "srv_count",
  "serror_rate",
  "srv_serror_rate",
  "rerror_rate",
  "srv_rerror_rate",
  "same_srv_rate",
  "diff_srv_rate",
  "srv_diff_host_rate",
  "dst_host_count",
  "dst_host_srv_count",
  "dst_host_same_srv_rate",
  "dst_host_diff_srv_rate",
  "dst_host_same_src_port_rate",
  "dst_host_srv_diff_host_rate",
  "dst_host_serror_rate",
...
res9_2: Int = 42

In [11]:
val data = dataNoHeader.toDF(nameArray:_*) // _* converts array to a varags (variable number of arguments)

data: DataFrame = [duration: int, protocol_type: string ... 40 more fields]

In [12]:
data.select("label").groupBy("label").count().orderBy(col("count").desc).show

show at cmd11.sc:1

8 / 8

show at cmd11.sc:1

8 / 8

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
+----------------+-------+
only showing top 20 rows



Create pipeline for the kmeans and run int

In [13]:
import $ivy.`org.apache.spark::spark-mllib:2.4.0`

import $ivy.$                                    

In [14]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.clustering.{KMeans, KMeansModel}

import org.apache.spark.ml.feature.VectorAssembler

In [15]:
data.getClass

res14: Class[T] = class org.apache.spark.sql.Dataset

In [16]:
data.schema

res15: types.StructType = StructType(
  StructField("duration", IntegerType, true, {}),
  StructField("protocol_type", StringType, true, {}),
  StructField("service", StringType, true, {}),
  StructField("flag", StringType, true, {}),
  StructField("src_bytes", IntegerType, true, {}),
  StructField("dst_bytes", IntegerType, true, {}),
  StructField("land", IntegerType, true, {}),
  StructField("wrong_fragment", IntegerType, true, {}),
  StructField("urgent", IntegerType, true, {}),
  StructField("hot", IntegerType, true, {}),
  StructField("num_failed_logins", IntegerType, true, {}),
  StructField("logged_in", IntegerType, true, {}),
  StructField("num_compromised", IntegerType, true, {}),
  StructField("root_shell", IntegerType, true, {}),
  StructField("su_attempted", IntegerType, true, {}),
  StructField("num_root", IntegerType, true, {}),
  StructField("num_file_creations", IntegerType, true, {}),
  StructField("num_shells", IntegerType, true, {}),
  StructField("num_access_files",

In [17]:
val numericalCols = data.drop("protocol_type","service","flag").cache

numericalCols: Dataset[Row] = [duration: int, src_bytes: int ... 37 more fields]

In [18]:
val assembler = new VectorAssembler().
                setInputCols(numericalCols.columns.filter(_ != "label")).
                setOutputCol("featureVector")

assembler: VectorAssembler = vecAssembler_be600441ab7f

Create null model with default options

In [19]:
val kmeans = new KMeans().
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")

kmeans: KMeans = kmeans_09a91fb5d571

Create pipeline for transforming ( assembel it into the input object expected of all ML alg), fit the model. This sets it up for hyper parameter tuning later using the spark.ml.tuning api

In [20]:
val pipeline = new Pipeline().setStages(Array(assembler,kmeans))
val pipelineModel = pipeline.fit(numericalCols)

takeSample at KMeans.scala:386

8 / 8

takeSample at KMeans.scala:386

8 / 8

sum at KMeans.scala:406

...

collect at ClusteringSummary.scala:51

8 / 8

collect at ClusteringSummary.scala:51

8 / 8

pipeline: Pipeline = pipeline_cf3a579ab770
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_cf3a579ab770

In [21]:
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
kmeansModel.clusterCenters.foreach(println)

[48.34019491959669,1834.6215497618625,826.2031900016945,5.7161172049003456E-6,6.487793027561892E-4,7.961734678254053E-6,0.012437658596734055,3.205108575604837E-5,0.14352904910348827,0.00808830584493399,6.818511237273984E-5,3.6746467745787934E-5,0.012934960793560386,0.0011887482315762398,7.430952366370449E-5,0.0010211435092468404,0.0,4.082940860643104E-7,8.351655530445469E-4,334.9735084506668,295.26714620807076,0.17797031701994256,0.1780369894027269,0.05766489875327379,0.05772990937912744,0.7898841322630906,0.02117961060991097,0.028260810096297884,232.98107822302248,189.21428335201279,0.7537133898007772,0.03071097882384052,0.605051930924901,0.006464107887636894,0.1780911843182284,0.1778858981346887,0.05792761150001272,0.05765922142401037]
[10999.0,0.0,1.309937401E9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,255.0,1.0,0.0,0.65,1.0,0.0,0.0,0.0,1.0,1.0]


kmeansModel: KMeansModel = kmeans_09a91fb5d571

In [22]:
val withCluster = pipelineModel.transform(numericalCols)

withCluster: DataFrame = [duration: int, src_bytes: int ... 39 more fields]

In [23]:
withCluster("cluster")

res22: Column = cluster

In [24]:
import spark.implicits._

import spark.implicits._

In [25]:
withCluster.select("cluster","label").groupBy("cluster","label").count()
                .orderBy($"cluster",$"count".desc).show(25)

show at cmd24.sc:2

8 / 8

show at cmd24.sc:2

8 / 8

+-------+----------------+-------+
|cluster|           label|  count|
+-------+----------------+-------+
|      0|          smurf.|2807886|
|      0|        neptune.|1072017|
|      0|         normal.| 972781|
|      0|          satan.|  15892|
|      0|        ipsweep.|  12481|
|      0|      portsweep.|  10412|
|      0|           nmap.|   2316|
|      0|           back.|   2203|
|      0|    warezclient.|   1020|
|      0|       teardrop.|    979|
|      0|            pod.|    264|
|      0|   guess_passwd.|     53|
|      0|buffer_overflow.|     30|
|      0|           land.|     21|
|      0|    warezmaster.|     20|
|      0|           imap.|     12|
|      0|        rootkit.|     10|
|      0|     loadmodule.|      9|
|      0|      ftp_write.|      8|
|      0|       multihop.|      7|
|      0|            phf.|      4|
|      0|           perl.|      3|
|      0|            spy.|      2|
|      1|      portsweep.|      1|
+-------+----------------+-------+



In [26]:
kmeansModel.extractParamMap

res25: org.apache.spark.ml.param.ParamMap = {
	kmeans_09a91fb5d571-distanceMeasure: euclidean,
	kmeans_09a91fb5d571-featuresCol: featureVector,
	kmeans_09a91fb5d571-initMode: k-means||,
	kmeans_09a91fb5d571-initSteps: 2,
	kmeans_09a91fb5d571-k: 2,
	kmeans_09a91fb5d571-maxIter: 20,
	kmeans_09a91fb5d571-predictionCol: cluster,
	kmeans_09a91fb5d571-seed: -1689246527,
	kmeans_09a91fb5d571-tol: 1.0E-4
}

Note that the default number of clusters is 2.

Because this is unsupervised there isn't an obvious loss function that can be used to choose optimal parameters such as in the case of regression where purity or entropy could be used. The loss function to be used in the sum of squared distance to the closest centres which intuitively should be minimised. However, this is not implemented in org.apache.spark.evaluators api and must be done manually

In [27]:
import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.DataFrame

In [28]:
import scala.util.Random

import scala.util.Random

def myClusterScore(data: DataFrame, k: Int): Double = {
    val assembler = new VectorAssembler().setInputCols(data.columns.filter(_!="label")).setOutputCol("featureVector")
    val kmeans = new KMeans().setFeaturesCol("featureVector").setSeed(Random.nextLong()).setK(k).setPredictionCol("cluster")
    
    val pipeline= new Pipeline().setStages(Array(assembler,kmeans))
    val kmeansModel=pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computeCost(assembler.transform(data))/data.count()
}

(20 to 100 by 20).map(k=>myClusterScore(numericalCols,k)).foreach(println)

Do a cross tab on the predicitons of the Kmeans model to see the 'purity' of the clusters

kmeansModel.extractParamMap

In [29]:
val preds = kmeansModel.transform(assembler.transform(numericalCols))

preds: DataFrame = [duration: int, src_bytes: int ... 39 more fields]

In [30]:
val labels = data.select("label").as[String].collect

collect at cmd29.sc:1

8 / 8

labels: Array[String] = Array(
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
...

In [31]:
val predCol = preds.select("cluster").as[Int].collect()

collect at cmd30.sc:1

8 / 8

predCol: Array[Int] = Array(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...

This code cell below seems to be problematic. It leads to GC overflow. Trying other ways wiht less GC overhead.

val xtab = spark.createDataset(labels.zip(predCol))

In [32]:
val xtab = labels.zip(predCol)

xtab: Array[(String, Int)] = Array(
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
...

In [37]:
val xtabLabeled= spark.createDataset(xtab)

xtabLabeled: Dataset[(String, Int)] = [_1: string, _2: int]

In [38]:
xtabLabeled.getClass

res37: Class[T] = class org.apache.spark.sql.Dataset

In [39]:
xtabLabeled.groupBy("_2").pivot("_1").count().show()

pivot at cmd38.sc:1

0 + 8 / 8

------------ UNHANDLED EXCEPTION ---------- (scala-kernel-zeromq-polling-1)
Exception in thread "dispatcher-event-loop-7" java.lang.OutOfMemoryError: GC overhead limit exceeded
	at cats.effect.internals.ArrayStack.<init>(ArrayStack.scala:36)
	at cats.effect.internals.ArrayStack.<init>(ArrayStack.scala:37)
	at cats.effect.internals.IORunLoop$.cats$effect$internals$IORunLoop$$loop(IORunLoop.scala:75)
	at cats.effect.internals.IORunLoop$.startCancelable(IORunLoop.scala:41)
	at cats.effect.internals.IORace$.$anonfun$simple$3(IORace.scala:85)
	at cats.effect.internals.IORace$.$anonfun$simple$3$adapted(IORace.scala:66)
	at cats.effect.internals.IORace$$$Lambda$762/2123216367.apply(Unknown Source)
	at cats.effect.internals.IORunLoop$RestartCallback.start(IORunLoop.scala:341)
	at cats.effect.internals.IORunLoop$.cats$effect$internals$IORunLoop$$loop(IORunLoop.scala:119)
	at cats.effect.internals.IORunLoop$RestartCallback.signal(IORunLoop.scala:355)
	at cats.effect.internals.IORunLoop$RestartCa

: 